In [2]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')

In [3]:
venues = venues.dropna()

In [4]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

In [5]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

In [6]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

Selected bus stops: ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']


In [7]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [8]:
merged_matrix.set_index('Unnamed: 0', inplace=True)
merged_matrix

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,3098.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,3209.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,4676.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,2214.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,2250.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,1234.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,2315.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0


In [9]:
print("Number of NaN values in merged_matrix:", merged_matrix.isna().sum().sum())

Number of NaN values in merged_matrix: 610


In [10]:
print("Rows with NaNs:")
print(merged_matrix[merged_matrix.isna().any(axis=1)])

print("Columns with NaNs:")
print(merged_matrix.columns[merged_matrix.isna().any()])


Rows with NaNs:
                A1     A10     A11     A12     A13     A14     A15     A16  \
Unnamed: 0                                                                   
A1             0.0  3304.0  3496.0  1727.0  1956.0  2020.0  2410.0  1512.0   
A10         3263.0     0.0  4901.0  2745.0  2974.0  3033.0  2460.0  2917.0   
A11         3459.0  4958.0     0.0  3305.0  3534.0  3598.0  4079.0  2432.0   
A12         1753.0  2784.0  3117.0     0.0   593.0  1014.0  1617.0  1405.0   
A13         1909.0  2939.0  3272.0   576.0     0.0   489.0  1654.0  1561.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
BT24        2357.0  2960.0  3911.0  1473.0  1509.0  1455.0  1027.0  2040.0   
BT25        2234.0  2575.0  3788.0  1350.0  1386.0  1332.0   250.0  1918.0   
BT26        1594.0  2406.0  3326.0  1783.0  1938.0  2233.0  1496.0  1365.0   
BT27        1485.0  2420.0  3408.0  1424.0  1579.0  1874.0  1556.0  1447.0   
BT28        1745.0  2006.0  3477.0  1630.0  1785

In [11]:
nan_positions = merged_matrix.isna()
print("NaN symmetric check (should be True for all):")
print((nan_positions == nan_positions.T).all())


NaN symmetric check (should be True for all):
A1      True
A10     True
A11     True
A12     True
A13     True
        ... 
BT24    True
BT25    True
BT26    True
BT27    True
BT28    True
Length: 154, dtype: bool


In [12]:
max_dist = merged_matrix.max().max()
fill_value = max_dist * 10
merged_matrix = merged_matrix.fillna(fill_value)
print("Filled merged_matrix:")
print(merged_matrix)

Filled merged_matrix:
                A1     A10     A11     A12     A13     A14     A15     A16  \
Unnamed: 0                                                                   
A1             0.0  3304.0  3496.0  1727.0  1956.0  2020.0  2410.0  1512.0   
A10         3263.0     0.0  4901.0  2745.0  2974.0  3033.0  2460.0  2917.0   
A11         3459.0  4958.0     0.0  3305.0  3534.0  3598.0  4079.0  2432.0   
A12         1753.0  2784.0  3117.0     0.0   593.0  1014.0  1617.0  1405.0   
A13         1909.0  2939.0  3272.0   576.0     0.0   489.0  1654.0  1561.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
BT24        2357.0  2960.0  3911.0  1473.0  1509.0  1455.0  1027.0  2040.0   
BT25        2234.0  2575.0  3788.0  1350.0  1386.0  1332.0   250.0  1918.0   
BT26        1594.0  2406.0  3326.0  1783.0  1938.0  2233.0  1496.0  1365.0   
BT27        1485.0  2420.0  3408.0  1424.0  1579.0  1874.0  1556.0  1447.0   
BT28        1745.0  2006.0  3477.0  1630.0

In [13]:
print("Highest value in merged_matrix:", merged_matrix.values.max())

Highest value in merged_matrix: 718410.0


In [14]:
# Extract rows corresponding to the selected bus stops
selected_bus_stops_matrix = merged_matrix.loc[selected]
selected_bus_stops_matrix

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BD14,2089.0,3107.0,3644.0,749.0,260.0,669.0,1754.0,1773.0,2633.0,2435.0,...,1851.0,1994.0,2391.0,2072.0,1279.0,1691.0,1572.0,1856.0,1727.0,1670.0
BD15,452.0,3066.0,3726.0,1556.0,1711.0,2006.0,2202.0,1765.0,3122.0,2924.0,...,1810.0,1953.0,2151.0,2560.0,1768.0,2180.0,2051.0,1553.0,1186.0,1629.0
BD16,2703.0,3916.0,2465.0,2362.0,2517.0,2814.0,3051.0,1624.0,3929.0,3732.0,...,2386.0,2671.0,2058.0,3369.0,2576.0,2987.0,2869.0,2415.0,2474.0,2364.0
BT16,2149.0,1620.0,3757.0,1666.0,1822.0,2116.0,1413.0,1796.0,1999.0,2208.0,...,431.0,708.0,1267.0,1470.0,1971.0,1979.0,1298.0,1153.0,1474.0,679.0
BT03,1878.0,3284.0,2537.0,1471.0,1627.0,1924.0,2160.0,624.0,3039.0,2842.0,...,1753.0,1942.0,1417.0,2479.0,1685.0,2097.0,1979.0,1685.0,1744.0,1666.0
BT08,1788.0,2801.0,3342.0,904.0,927.0,873.0,972.0,1471.0,1851.0,1653.0,...,1545.0,1688.0,2089.0,1290.0,497.0,909.0,790.0,1550.0,1421.0,1364.0
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
BL14,2279.0,2498.0,3992.0,1553.0,1590.0,1536.0,325.0,2210.0,1235.0,1027.0,...,1164.0,1276.0,1685.0,709.0,1061.0,798.0,251.0,1282.0,1517.0,952.0
BD04,3152.0,2792.0,4706.0,2268.0,2304.0,2250.0,1349.0,2836.0,666.0,1403.0,...,1808.0,1934.0,2558.0,1297.0,1775.0,1387.0,1255.0,2286.0,2521.0,1909.0


In [15]:
# Extract the indices of the selected bus stops from the merged matrix
selected_indices = merged_matrix.index.get_indexer(selected_bus_stops_matrix.index)
selected_indices

array([ 98,  99, 100, 141, 128, 133, 149, 115,  88, 136, 131, 146, 121,
       143, 124])

In [34]:
# Create a list of pickup and delivery pairs
pickup_delivery_pairs = [(accomodation_id, venue_id) for accomodation_id in accomodations_clusters['id'] for venue_id in venues['id']]

# Display the list of pairs
pickup_delivery_pairs
pickup_delivery_pairs = pickup_delivery_pairs[:7]  # test with 10 requests


# VRP

In [35]:
#import pandas as pd

# Your selected bus stops:
#selected_stops = ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24',
                  #'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']

# Assume merged_matrix is your original big DataFrame

# Filter rows and columns to keep only selected stops
#reduced_matrix = merged_matrix.loc[selected_stops, selected_stops].copy()

In [36]:
##max_dist = reduced_matrix.max().max()
#fill_value = max_dist * 10
#reduced_matrix = reduced_matrix.fillna(fill_value)
#print("Filled merged_matrix:")
#print(reduced_matrix)

In [37]:
#import pandas as pd

# Replace 'your_username' with your actual username or provide the full path to the file
#new_matrix_path = '/Users/your_username/Downloads/new_matrix.csv'

# Load the CSV file into a DataFrame
#new_matrix = pd.read_csv(new_matrix_path)

# Display the first few rows of the DataFrame
#new_matrix.head()

In [38]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np

In [39]:
import pandas as pd

# List of selected bus stops
selected_stops = ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 
                  'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']

# Suppose `merged_matrix` is a pandas DataFrame where rows and columns are stop names
# You want to keep only the selected rows and columns

filtered_matrix = merged_matrix.loc[selected_stops, selected_stops].copy()

In [40]:
def create_data_model():
    data = {}
    data['distance_matrix'] = merged_matrix.values.tolist()
    data['num_vehicles'] = 7
    
    # These are the actual node indices in the full matrix (from FLP)
    data['starts'] = [98, 99, 100, 141, 128, 133, 149]
    data['ends'] = data['starts']  # if routes must return to starting depot
    
    data['pickups_deliveries'] = pickup_delivery_pairs
    data['venues'] = set(venues)
    
    return data

data = create_data_model()


In [41]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['starts'], data['ends'])
routing = pywrapcp.RoutingModel(manager)

In [42]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    if from_index < 0 or from_index >= routing.Size():
        print(f"Invalid from_index: {from_index} (routing size {routing.Size()})")
        raise ValueError("from_index out of range")
    if to_index < 0 or to_index >= routing.Size():
        print(f"Invalid to_index: {to_index} (routing size {routing.Size()})")
        raise ValueError("to_index out of range")

    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)

    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [43]:
print(f"Number of nodes (manager): {manager.GetNumberOfNodes()}")
print(f"Routing model size: {routing.Size()}")
print(f"Distance matrix size: {len(data['distance_matrix'])}")


Number of nodes (manager): 154
Routing model size: 154
Distance matrix size: 154


In [44]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [45]:
# Create a mapping for non-integer identifiers
node_mapping = {node: idx for idx, node in enumerate(set(sum(pickup_delivery_pairs, ())))}

# Convert pickup_delivery_pairs to use integer indices
data['pickups_deliveries'] = [
    (node_mapping[pickup], node_mapping[delivery]) for pickup, delivery in pickup_delivery_pairs
]

# Add pickup and delivery constraints
for pickup, delivery in data['pickups_deliveries']:
    pickup_index = manager.NodeToIndex(pickup)
    delivery_index = manager.NodeToIndex(delivery)
    routing.AddPickupAndDelivery(pickup_index, delivery_index)

In [46]:
print(f"Routing size: {routing.Size()}")
print(f"Number of nodes (manager): {manager.GetNumberOfNodes()}")
print(f"Distance matrix size: {len(data['distance_matrix'])} x {len(data['distance_matrix'][0])}")
print(f"Vehicle start indices: {data['starts']}")
print(f"Vehicle end indices: {data['ends']}")


Routing size: 154
Number of nodes (manager): 154
Distance matrix size: 154 x 154
Vehicle start indices: [98, 99, 100, 141, 128, 133, 149]
Vehicle end indices: [98, 99, 100, 141, 128, 133, 149]


In [47]:
#Ensure all nodes are visited
penalty = 100000  # high enough to force consideration
for node in range(1, len(data['distance_matrix'])):
    if node not in data['starts'] and node not in data['ends']:
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

In [ ]:
#limit each route to do max 25 stops
routing.AddConstantDimension(
    1,  # Increment by 1 per stop
    25,  # Maximum number of stops per vehicle
    True,  # Start cumul at zero
    'StopCount'
)

# (Optional) Get the dimension to enforce additional constraints if needed
stop_count_dimension = routing.GetDimensionOrDie('StopCount')

: 

In [ ]:
#solver
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.SAVINGS)

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

search_parameters.time_limit.seconds = 30

solution = routing.SolveWithParameters(search_parameters)

#print solution
def print_solution(data, manager, routing, solution):
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            plan_output += f' {node} ->'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node = manager.IndexToNode(index)
        plan_output += f' {node}\nDistance of the route: {route_distance}m\n'
        print(plan_output)
        total_distance += route_distance
    print(f'Total distance of all routes: {total_distance}m')

if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found.")


In [ ]:
# Create a map centered at the average latitude and longitude of the venues
map_center = [venues['Latitude'].mean(), venues['Longitude'].mean()]
route_map = folium.Map(location=map_center, zoom_start=10)

# Add smaller markers for each venue
for _, row in venues.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,  # Smaller radius for the marker
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"Venue: {row['Venue']}<br>ID: {row['id']}<br>Capacity: {row['Approx. Capacity']}"
    ).add_to(route_map)

# Add smaller markers for each selected bus stop
for stop_id in selected_stops:
    stop_row = bus_terminals[bus_terminals['id'] == stop_id].iloc[0]
    folium.CircleMarker(
        location=[stop_row['Latitude'], stop_row['Longitude']],
        radius=5,  # Smaller radius for the marker
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        popup=f"Bus Stop: {stop_row['FACILITY']}<br>ID: {stop_row['id']}"
    ).add_to(route_map)

# Draw routes for each vehicle
colors = ['red', 'purple', 'orange', 'darkblue', 'darkgreen']  # Colors for routes
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    route_coordinates = []
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        if node < len(selected_stops):  # Check if it's a bus stop
            stop_id = selected_stops[node]
            stop_row = bus_terminals[bus_terminals['id'] == stop_id].iloc[0]
            route_coordinates.append([stop_row['Latitude'], stop_row['Longitude']])
        index = solution.Value(routing.NextVar(index))
    
    # Add the route to the map if it has coordinates
    if route_coordinates:
        folium.PolyLine(
            route_coordinates,
            color=colors[vehicle_id % len(colors)],  # Cycle through colors
            weight=5,
            opacity=0.8
        ).add_to(route_map)
    else:
        print(f"Vehicle {vehicle_id} has no assigned route.")

# Display the map
route_map

AttributeError: 'NoneType' object has no attribute 'Value'